# Importation Bibliothèque

In [1]:
import pandas as pd
import numpy as np
import json

from time import time
from collections import Counter

# Creation des dictionnaires pour la recommendation

## Dictionnaire pour les genres musicaux

In [2]:
# Chargement du fichier csv musique_spotify et suppression des lignes n'ayant pas de genres musicaux
df=pd.read_csv('Data/musique_spotify.csv',index_col=0)
df=df[['id','genre','popularity']].dropna()
df.head()

,id,genre,popularity
0,7lmeHLHBe4nmXzuXc0HDjk,"alternative metal, alternative rock, conscious...",65
1,1wsRitfRRtWyEapl0q22o8,"alternative metal, alternative rock, conscious...",69
2,1hR0fIFK2qRG3f3RF70pb7,"alternative metal, alternative rock, conscious...",55
3,2lbASgTSoDO7MTuLAXlTW0,"alternative metal, alternative rock, conscious...",50
4,1MQTmpYOZ6fcMQc56Hdo7T,"alternative metal, alternative rock, conscious...",66


In [3]:
# Création d'un liste contenant toute les lignes de la colonne "genre" de df
genre_mu=''
for genre in df['genre']:
    genre_mu+=' '+genre
    
liste_genre=genre_mu[1:-1].split(', ')



In [4]:
# A partir de la liste fabriqué précedemment, 
# nous avons crée avec à l'aide de la fonction Counter de la bibliothèque collections 
# un DataFrame avec les genres musicaux et leur nombre de répétition dans le DataFrame df

Nbr_genre=pd.DataFrame({'genre':list(Counter(liste_genre).keys()),
                        'Nombre':list(Counter(liste_genre).values())
                       })
Nbr_genre=Nbr_genre.sort_values(by='Nombre',ascending=False)
print("Il y a",Nbr_genre.shape[0],'genres musicales différents')
Nbr_genre.head(10)


Il y a 4696 genres musicales différents


,genre,Nombre
104,classical,88721
8,rock,28635
121,baroque,27626
122,early music,27569
125,early modern classical,24951
11,mellow gold,23479
12,soft rock,22480
10,classic rock,21779
264,classical era,19144
21,country rock,19079


In [5]:
# Nous supprimons les genres n'apparaisant qu'une fois qui serons inutile pour la recommendation
Nbr_genre=Nbr_genre[Nbr_genre['Nombre']>1]
print("Il y a",Nbr_genre.shape[0],'genres musicales différents avec au moins deux ittérations')

Il y a 4518 genres musicales différents avec au moins deux ittérations


In [6]:
## Création d'une liste contenant les différents genre musicaux
liste_genre=list(Nbr_genre['genre'])

In [7]:
# Création d'une dictionnaire donnant pour un genre musical les musiques associé dans df
genre2music={}
for str_genre,music_g in zip (df['genre'],df['id']):
    genres=str_genre[:-1].split(', ')
    for i in genres:
        if i not in genre2music:
            if i in liste_genre:
                genre2music[i]=[music_g]
        else:
            genre2music[i].append(music_g)

In [8]:
# Création d'une dictionnaire donnant pour une musique les genres musicaaux associé dans df
music2genre={}
for str_genre,music_g in zip (df['genre'],df['id']):
    genres=str_genre[:-1].split(', ')
    music2genre[music_g]=[]
    for genre in genres:
        if genre in liste_genre:
            music2genre[music_g].append(genre)
    if music2genre[music_g]==[]:
        music2genre.pop(music_g)

In [9]:
# Création d'une dictionnaire donnant pour une musique les genres musicaaux associé dans df
music2popularity={}
for music,popularity in zip(df['id'],df['popularity']):
    music2popularity[music]=popularity

In [10]:
# Liste des musiques présente dans nos dictionnaire
liste_music=list(music2popularity.keys())

In [11]:
# Création d'une dictionnaire contenant toute les donnée à sauvegarder pour la recommendation
Recommendation_genre={'genre2music':genre2music,
                      'music2genre':music2genre,
                      'music2popularity':music2popularity,
                      'liste_music':liste_music    
}

In [12]:
##Enregistrement des données
tf = open("Data/Recommendation_genre.json", "w")
json.dump(Recommendation_genre,tf)
tf.close()

## Dictionnaire pour les playlists sur les musiques

In [13]:
df=pd.read_csv('Data/playlist_musique_spotify.csv',index_col=0)
df.head()

,playlist,id
0,0,6I9VzXrHxO9rA9A5euc8Ak
1,0,1AWQoqb9bSvzTjaLralEkT
2,0,7H6ev70Weq6DdpZyyTmUXk
3,0,2PpruBYCo4H7WOBJ7Q2EwM
4,0,4pmc2AxSEq6g7hPVlJCPyP


In [14]:
# Supression des musiques présente plusieurs fois dans la même playlist
df=df.drop_duplicates()

In [15]:
# Creation des dictionnaires reliant les playlists aux musiques qu'il contiennent
# et les musiques au playlist les contenants
playlist2music={}
music2playlist={}
for playlist, track in zip(df['playlist'],df['id']):
    if playlist not in playlist2music:
        playlist2music[playlist]=[track]
    else:
        playlist2music[playlist].append(track)
        
    if track not in music2playlist:
        music2playlist[track]=[playlist]
    else:
        music2playlist[track].append(playlist)

In [16]:
# Liste des musiques présente dans nos dictionnaire
liste_music=list(df['id'].unique())

In [17]:
# Création d'une dictionnaire contenant toute les donnée à sauvegarder pour la recommendation
Recommendation_playlist_musique={'playlist2music':playlist2music,
                                 'music2playlist':music2playlist,
                                 'liste_music':liste_music}

In [18]:
##Enregistrement des données
tf = open("Data/Recommendation_playlist_musique.json", "w")
json.dump(Recommendation_playlist_musique,tf)
tf.close()

## Dictionnaire pour le dataset playlist

In [19]:
df=pd.read_csv('Data/playlist_spotify_spare.csv',index_col=0)
df.head()

,playlist,track_uri
0,0,0UaMYEvWZi0ZqiDOoHU3YI
1,0,6I9VzXrHxO9rA9A5euc8Ak
2,0,0WqIKmW4BTrj3eJFmnCKMv
3,0,1AWQoqb9bSvzTjaLralEkT
4,0,1lzr43nnXAijIGYnCT8M8H


In [20]:
# Supression des musiques présente plusieurs fois dans la même playlist
df=df.drop_duplicates()

In [21]:
# Creation des dictionnaires reliant les playlists aux musiques qu'il contiennent
# et les musiques au playlist les contenants
playlist2music={}
music2playlist={}
for playlist, track in zip(df['playlist'],df['track_uri']):
    if playlist not in playlist2music:
        playlist2music[playlist]=[track]
    else:
        playlist2music[playlist].append(track)
        
    if track not in music2playlist:
        music2playlist[track]=[playlist]
    else:
        music2playlist[track].append(playlist)

In [22]:
# Liste des musiques présente dans nos dictionnaire
liste_music=list(df['track_uri'].unique())

In [23]:
# Création d'une dictionnaire contenant toute les donnée à sauvegarder pour la recommendation
Recommendation_playlist={'playlist2music':playlist2music,
                                 'music2playlist':music2playlist,
                                 'liste_music':liste_music}

In [24]:
##Enregistrement des données
tf = open("Data/Recommendation_playlist.json", "w")
json.dump(Recommendation_playlist,tf)
tf.close()